# Create vocab

## We create vocab for the models by combining all the train dataset.
Since we work with multiple datasets and building vocab based on them, it makes sense to build a combined vocab to be used for all training and inference

In [2]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from utils.early_stopping import EarlyStopping
import numpy as np
import copy
from tqdm import tqdm
from model.ha import HierarchicalAttPredictor
from sklearn.metrics import classification_report
# from data.evaluate import load_dev_labels, get_metrics
import pickle as pkl
import sys
from allennlp.modules.elmo import Elmo, batch_to_ids
from copy import deepcopy
import argparse
import random
from utils.focalloss import FocalLoss
from torchmoji.sentence_tokenizer import SentenceTokenizer
from torchmoji.global_variables import PRETRAINED_PATH, VOCAB_PATH
from utils.tweet_processor import processing_pipeline
import json
import pandas as pd
from nltk.tokenize import sent_tokenize
from module.preprocessor import EnglishPreProcessor
from pytorch_pretrained_bert.tokenization import BertTokenizer
from config.basic_config import configs as config
import re
import seaborn as sns

 stats_dir /home/kathleenwang/.ekphrasis/stats
Reading twitter - 1grams ...
 stats_dir /home/kathleenwang/.ekphrasis/stats
Reading twitter - 2grams ...
 stats_dir /home/kathleenwang/.ekphrasis/stats
Reading twitter - 1grams ...


In [2]:
from module import create_data

In [3]:
# proc pipeline not working, have to remove it
def load_data_context(data_path='/data/SuperMod/test_data.txt', is_train=True):

    data_list = []
    target_list = []
    
    df = pd.read_csv(data_path)
    
    if len(df.columns) > 4:
        data_list = df.comment_text.tolist()
        target_list = df.toxic.tolist()   

    else:
        data_list = df.comment_text.tolist()
        target_list = df.toxicity.tolist()

    clean_sent_list = [sent_tokenize(processing_pipeline(str(email))) for email in data_list]

    if is_train:
        return clean_sent_list, target_list
    else:
        return clean_sent_list
   


In [4]:
# import nltk
# nltk.download('punkt')

In [5]:
wiki_train = pd.read_csv('/data/SuperMod/train.csv')

In [6]:
# use test_with_label instead of the actual test because not all test has label
wiki_test = pd.read_csv('/data/SuperMod/test_with_labels.csv')

In [7]:
train_s, label_s = load_data_context(data_path='/data/SuperMod/training_data_supermod.csv')

In [8]:
train_t, label_t = load_data_context(data_path='/data/SuperMod/train.csv')

In [9]:
train_v, label_v = load_data_context(data_path='/data/SuperMod/test_with_labels.csv')

In [10]:
train_mt, label_mt = load_data_context(data_path="/data/SuperMod/imdb_test.csv")

In [11]:
train_m, label_m = load_data_context(data_path="/data/SuperMod/imdb_train.csv")

In [12]:
word2id, id2word, num_of_vocab = create_data.build_vocab([train_s, train_t, train_v, train_mt, train_m], 20000, True)

found 24205 words
filling vocab to 20003


In [13]:
with open('/data/SuperMod/word2id.pkl', 'wb') as w:
    pkl.dump(word2id, w, pkl.HIGHEST_PROTOCOL)
with open('/data/SuperMod/id2word.pkl', 'wb') as i:
    pkl.dump(id2word, i, pkl.HIGHEST_PROTOCOL)

In [15]:
wiki_mini = wiki_train[:100]
wiki_mini.to_csv('/data/SuperMod/train_mini.csv')

## test out train script

In [16]:
!python train_ha.py -folds 2 -epoch 2 -input_path '/data/SuperMod/train_mini.csv' -test_path '/data/SuperMod/train_mini.csv'  -out_path '/data/SuperMod/hapy_state_delete.pth'

 stats_dir /home/kathleenwang/.ekphrasis/stats
Reading twitter - 1grams ...
 stats_dir /home/kathleenwang/.ekphrasis/stats
Reading twitter - 2grams ...
 stats_dir /home/kathleenwang/.ekphrasis/stats
Reading twitter - 1grams ...
Tokenizing using dictionary from /data/torchMoji/model/vocabulary.json
creating pkl file for the emb text file
Loading Glove Model
100%|██████████████████████████████| 2196017/2196017 [02:21<00:00, 15480.68it/s]
Done. 2196016  words loaded!
loading glove
100%|█████████████████████████████████| 20003/20003 [00:00<00:00, 377177.74it/s]
15614 of 20003 found coverage 0.7805829125631155
Size of final test data 100
Train size: 50 Dev size: 50
Training on fold: 0
Loading weights for embed.weight
Loading weights for lstm_0.weight_ih_l0
Loading weights for lstm_0.weight_hh_l0
Loading weights for lstm_0.bias_ih_l0
Loading weights for lstm_0.bias_hh_l0
Loading weights for lstm_0.weight_ih_l0_reverse
Loading weights for lstm_0.weight_hh_l0_reverse
Loading weights for lstm_0